# IR Swap

Interest rate swap (IRS) is a financial product to exchange future interest payments, when one party (payer) agrees to pay a fixed rate, second party (receiver) agrees to pay a floating rate (usually tied to LIBOR or some other index) for a notional amount and for an agreed period of time.

Notes:

*Payment dates*: agreed, but party which pays fixed rate usually pays is less frequent than party which pays the floating rate.

*Basis swap*: when both parties agree to exchange a floating rate.

*Future contract*: IRS is an Over-The-Counter (OTC) contract between two (or more) counterparties.

*Investments are not swaped*: only interest rate payments are, usually netted.

In [1]:
#!fsharp
let days = 360
let rate = 0.05
let notional = 100_000.
let dailyRate = rate / float days

In [1]:
#!fsharp
open System

let generateOvernightRateCurve (seed : double) = 
    let rand = Random()
    Seq.initInfinite (fun i -> i)
    |> Seq.scan (fun acc _ -> acc + (float (rand.Next(-100, 100)) / 10_000.)) seed
    |> Seq.mapi (fun i value -> i + 1, value)

In [1]:
#!fsharp
let calculateInterest (notional : float) (rate : float) = notional * rate

In [1]:
#!fsharp
let calculateCompounded period notional rate = 
    let dailyRate = rate / 360.
    [1..period]
    |> List.scan (fun acc _ -> acc + (calculateInterest acc dailyRate) ) notional

let calculateFixed period notional rate = 
    let dailyRate = rate / 360.
    [1..period]
    |> List.map (fun _ -> (calculateInterest notional rate) + notional )

In [1]:
#!fsharp
let calculateWithDaily period notional (rateMap : Map<int,float>) = 
    [1..period]
    |> List.scan (fun acc i -> 
        let dailyRate = rateMap.[i] / 360.
        acc + (calculateInterest notional dailyRate)
    ) notional

In [1]:
#!fsharp
#r "nuget:XPlot.Plotly"

Installed package XPlot.Plotly version 3.0.1

In [1]:
#!fsharp
open XPlot.Plotly

In [1]:
#!fsharp
let rates = generateOvernightRateCurve 0.05 |> Seq.truncate days |> Map.ofSeq
let curve = calculateWithDaily days notional rates 

Chart.Line curve

In [1]:
#!fsharp
let data = calculateFixed 360 notional 0.06
Chart.Line data

In [1]:
#!fsharp
